In [1]:
import os
import pickle
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity

In [2]:
from srcfanova.confspace_utils import get_configspace, integer_encode_dataframe


import itertools as it
from collections import OrderedDict

import ConfigSpace
import numpy as np
import pandas as pd
import pyrfr.regression as reg
import pyrfr.util
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, \
    NumericalHyperparameter, Constant, OrdinalHyperparameter

from surrogate import fANOVA_surrogate
from hyperband_infinite import HyperBandOptimiser

from typing import List

# Prepare data for fitting surrogate models 

In [3]:
dataf = pd.read_csv('./results_hyper.csv', sep=",")

# removing these two datasets because the performance can not be explained using ANOVA
dataf = dataf[dataf.dataset != 'ilpd']
dataf = dataf[dataf.dataset != 'blood-transfusion-service-center']
dataf = dataf.reset_index()
task_ids = sorted(dataf['task_id'].unique())

measure = 'val_binary_accuracy'

# important hyperparameter keys to consider: learning_rate, depth, use_reuploading, input_activation_function
# make sure data is numerical and in right order for configspace
config_space = get_configspace(bool(1))
cs_params = config_space.get_hyperparameter_names()

original_df = dataf.loc[:, [cs_params[i] for i in range(len(cs_params))]]

data = dataf.loc[:, [cs_params[i] for i in range(len(cs_params))]]
data = integer_encode_dataframe(data, config_space)
data['task_id'] = dataf.task_id
data['dataset'] = dataf.dataset
data[measure] = dataf[measure]

# Fit surrogate models

In [5]:
model_per_task = {}
n_trees= 128

for t_idx, task_id in enumerate(task_ids):
    
    data_task = data[data['task_id'] == task_id]
    del data_task['task_id']
    del data_task['dataset']
    
    y_data = data_task[measure].values
    X_data = data_task.copy()
    del X_data[measure]
    
    model_per_task[task_id] = fANOVA_surrogate(X=X_data, Y=y_data, n_trees=n_trees, seed=t_idx)

# Hyperband Optimisation Uniform vs KDE Prior Experiment

The experiment configuration for the uniform prior experiment is defined in the next cell.

In [6]:
n_runs = 15

seed_exp = np.arange(n_runs)
eta_exp = [2, 3, 4] # halving factor
starting_budget_exp = [int(1e3), int(1e4)] # starting budget, can be though of as number of shots to compute the expectation value.
max_k_exp = [4, 5, 6] # defines how many iterations does the hyperband algorithm run for; also called s_max in the paper.
search_type = 'uniform' 

# Run hyperband algorithm with uniform priors

Doing a search over hyperparameter configuration space by employing hyperband with uniform sampling of hyperparameters. The objective is to find the hyperparameter configuration which gives maximum validation accuracy of the surrogate model.

In [ ]:
# For uniform prior experiments

for max_k in max_k_exp:
    for eta in eta_exp:
        for starting_budget in starting_budget_exp:
            for task_id in task_ids:
                for seed in seed_exp:
                    optimiser = HyperBandOptimiser(eta=eta,
                                                   config_space=config_space,
                                                   optimisation_goal='performance',
                                                   max_k=max_k,
                                                   min_or_max=max,
                                                   task_id=task_id,
                                                   starting_shots_nb=starting_budget,
                                                   search_type=search_type,
                                                   seed_nb=seed)
                    optimiser.run_optimisation(model_for_task_id=model_per_task[task_id],
                                               min_configs=eta,
                                               all_data=data,
                                               store_optimiser=True,
                                               verbosity=False)

# Retrieve & Save necessary results  

In [ ]:

for max_k in max_k_exp:
    for eta in eta_exp:
        for starting_budget in starting_budget_exp:
            
            results = {}
            for task_id in task_ids:
                results[task_id] = {}
                
            for task_id in task_ids:
                for seed in seed_exp:
                    f_name = f'./optimiser/{search_type}/task_id{task_id}_search_{search_type}_eta_{eta}_max_k_{max_k}_shots_{starting_budget}_seed_{seed}.pckl'
                    optimiser = pickle.load(open(f_name, 'rb'))
                    results[task_id][seed] = optimiser.eval_history
                    
                    f_name = f'./data/{search_type}_eta_{eta}_max_k_{max_k}_shots_{starting_budget}.pckl'
                    os.makedirs(os.path.dirname(f_name), exist_ok=True)
                    pickle.dump(results, open(f_name, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
                    

# Run hyperband algorithm with KDE priors

The experiment configuration for the uniform prior experiment is defined in the next cell.

In [ ]:
n_runs = 15

best_N_exp = [10, 20]
seed_exp = np.arange(n_runs)
eta_exp = [2, 3, 4]
starting_budget_exp = [int(1e3), int(1e4)]
max_k_exp = [4, 5, 6]
search_type = 'kde'

# this is given as a list of a list, where the inside list contains the indices of (important) hyperparameters
# Index {6: learning_rate, 1: depth, 4: input_activation_function, 9: use_reuploading}
imp_hyperparams_list_exp = [[6], [6, 1], [6, 1, 4], [6, 1, 4, 9]]
kde_bw_estimator_exp = ['sj', 'silverman'] # bandwith estimator for Kernel Density to fit the data

Doing a search over hyperparameter configuration space by employing hyperband with (some, mostly important ones) hyperparameters sampled from kernel density estimator which is fitted with best_N performing hyperparameter configurations for each task.

In [ ]:
# For kde prior experiments

for imp_hyperparams_list in imp_hyperparams_list_exp:
    for max_k in max_k_exp:
        for eta in eta_exp:
            for starting_budget in starting_budget_exp:
                for best_N in best_N_exp:
                    for kde_bw_estimator in kde_bw_estimator_exp:
                        for task_id in task_ids:
                            for seed in seed_exp:
                                optimiser = HyperBandOptimiser(eta=eta,
                                                               config_space=config_space,
                                                               optimisation_goal='performance',
                                                               max_k=max_k,
                                                               min_or_max=max,
                                                               task_id=task_id,
                                                               starting_shots_nb=starting_budget,
                                                               search_type=search_type,
                                                               important_hyperparams_indices=imp_hyperparams_list,
                                                               best_N=best_N,
                                                               seed_nb=seed,
                                                               kde_bw_estimator=kde_bw_estimator,
                                                               kde_bw=None,
                                                               pickle_path=None)
                                optimiser.run_optimisation(model_for_task_id=model_per_task[task_id],
                                                           min_configs=eta,
                                                           all_data=data,
                                                           store_optimiser=True,
                                                           verbosity=False)

# Retrieve & Save necessary results  

In [ ]:

for imp_hyperparams_list in imp_hyperparams_list_exp:
    for max_k in max_k_exp:
        for eta in eta_exp:
            for starting_budget in starting_budget_exp:
                for best_N in best_N_exp:
                    for kde_bw_estimator in kde_bw_estimator_exp:
                        
                        results = {}
                        for task_id in task_ids:
                            results[task_id] = {}
                        
                        for task_id in task_ids:
                            for seed in seed_exp:
                                opt_f_name = f'./optimiser/{search_type}/task_id{task_id}_search_{search_type}_bw_None_bw_est_{kde_bw_estimator}_bestN_{best_N}_eta_{eta}_max_k_{max_k}_shots_{starting_budget}_imp_hyp_{imp_hyperparams_list}_seed_{seed}.pckl'
                                optimiser = pickle.load(open(opt_f_name, 'rb'))
                                results[task_id][seed] = optimiser.eval_history
                        f_name = f'./data/{search_type}_bw_est_{kde_bw_estimator}_bestN_{best_N}_eta_{eta}_max_k_{max_k}_shots_{starting_budget}_imp_hyp_{imp_hyperparams_list}.pckl'
                        os.makedirs(os.path.dirname(f_name), exist_ok=True)
                        pickle.dump(results, open(f_name, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
                        

# Plot that Data Darn It!

In [ ]:
data_directory = './data/'
df = pd.DataFrame(columns=['task_id', 'seed', 'max_k', 'eta', 'starting_budget', 'imp_hyperparams', 'best_N', 'bw_estimator', 'mean_result_kde', 'mean_result_uniform', 'difference'])

In [ ]:

for imp_hyperparams_list in imp_hyperparams_list_exp:
    for max_k in max_k_exp:
        for eta in eta_exp:
            for starting_budget in starting_budget_exp:
            
                f_uni_name = data_directory + f'uniform_eta_{eta}_max_k_{max_k}_shots_{starting_budget}.pckl'
                uni_results = pickle.load(open(f_uni_name, 'rb'))
            
                for best_N in best_N_exp:
                    for kde_bw_estimator in kde_bw_estimator_exp:
                        data = []                                
                        f_kde_name = data_directory + f'kde_bw_est_{kde_bw_estimator}_bestN_{best_N}_eta_{eta}_max_k_{max_k}_shots_{int(starting_budget)}_imp_hyp_{imp_hyperparams_list}.pckl'
                        kde_results = pickle.load(open(f_kde_name, 'rb'))
                        
                        for task_id in task_ids:
                            for seed in seed_exp:

                                # we do this by taking either taking the max, median or mean of eval_history
                                scores_kde = np.mean(kde_results[task_id][seed])
                                scores_uniform = np.mean(uni_results[task_id][seed])
                                current_difference = scores_kde - scores_uniform
                                data.append(current_difference)
                                
                                current_row = {'task_id': task_id, 'seed': seed, 'max_k': max_k, 'eta': eta, 'starting_budget': starting_budget, 'imp_hyperparams': imp_hyperparams_list, 'best_N': best_N, 'bw_estimator': kde_bw_estimator, 'mean_result_kde': scores_kde, 'mean_result_uniform': scores_uniform, 'difference': current_difference}
                                df = df.append(current_row, ignore_index=True)
                        
                        plt.figure(figsize=(3, 6))
                        plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
                        plt.plot([0.5, 1.5], [0, 0], 'k-', linestyle='--', lw=1)
                        plt.violinplot(data)

                        plt.xlabel(f'{kde_bw_estimator}_{best_N}N_{eta}eta_{max_k}k_{int(starting_budget)}s_{imp_hyperparams_list}', fontweight='bold')
                        plt.tight_layout()
                        plot_name = f'./plots/uniform_v_kde_bw_est_{kde_bw_estimator}_bestN_{best_N}_eta_{eta}_max_k_{max_k}_shots_{int(starting_budget)}_imp_hyp_{imp_hyperparams_list}.pdf'
                        os.makedirs(os.path.dirname(plot_name), exist_ok=True)
                        plt.savefig(plot_name, bbox_inches='tight')
                        plt.clf()
                        